Check if you are in the right environment

In [ ]:
# ! at start of line means its running interminal (bash) not python
!conda env list
# you shoud see a * next to velocityenv

Set paths to the required fiels described in the README

In [ ]:
# we will set variables in this box
# %env at start of lines saves the variable to the environment
#path to 10x run folder
%env tx=./Desktop/20201110_10x2_WM983B_NoDrug_seq1/10X2_seq1_NoDrugWM983B_count/
#path to cellranger reference genome (can find them here: https://support.10xgenomics.com/single-cell-gene-expression/software/downloads/latest)
%env rgen=./Desktop/velocity_files/genes.gtf
#path to repeat masker in GTF format (can find here: https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=611454127_NtvlaW6xBSIRYJEBI0iRDEWisITa&clade=mammal&org=Human&db=0&hgta_group=allTracks&hgta_track=rmsk&hgta_table=rmsk&hgta_regionType=genome&position=&hgta_outputType=gff&hgta_outFileName=mm10_rmsk.gtf)
%env rpmsk=./Desktop/velocity_files/h38_rmsk.gtf

# if you have any "permission denied" error just go in the terminal activate the environment and use the chmod -R 775 <path to folder>
# for example: chmod -R 775 ./Desktop/20201110_10x2_WM983B_NoDrug_seq1/10X2_seq1_NoDrugWM983B_count/


Sort bam file using samtools

In [ ]:
#this creates a new sorted bam file named "cellsorted_possorted_genome_bam.bam" in the outs folder of 10x
!samtools sort -t CB -O BAM -m 5G -@ 4 -o $tx/outs/cellsorted_possorted_genome_bam.bam $tx/outs/possorted_genome_bam.bam

Run velocyto to get .loom file of exon and intron reads

In [ ]:
!velocyto run10x -m $rpmsk $tx $rgen
#after this is done running move to R if you did pre-processing using Seurat

Import packages and set scvelo settings

In [ ]:
#If loom file is already made start here
import scvelo as scv
import numpy as np
import matplotlib as plt
import os
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')

Load .h5ad file created in R

In [ ]:
#load loom file
#load path vraible into python
ptx=os.environ.get('tx') + '/velocyto/SeuratProcessed.h5ad'
adata = scv.read(ptx)
adata

Run all following block of code sequentially to get velocity outputs

In [ ]:
#see proportion of read types
scv.pl.proportions(adata)

In [ ]:
#Basic pre-processing using values given by suerat (http://htmlpreview.github.io/?https://github.com/satijalab/seurat-wrappers/blob/master/docs/scvelo.html)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
#calculate velocities
scv.tl.velocity(adata)
scv.tl.velocity_graph(adata)

In [ ]:
#visualize velocity (look at website for all options)
scv.pl.velocity_embedding_stream(adata, basis='umap', color="Phase",figsize= (12,8))
scv.pl.velocity_embedding(adata, basis='umap', color="seurat_clusters",arrow_length=6, arrow_size=8, dpi=120,figsize= (12,8))
scv.pl.velocity(adata,["FN1","AXL"], color = "seurat_clusters",ncols=1, basis = "umap", figsize= (24,16), size= 100)

At this point it should be straightforward to exectue any analysis described by scvelo here:https://scvelo.readthedocs.io/getting_started.html